# Chapter 12: Signaling failure

In [1]:
notThe :: String -> Maybe String
notThe "the" = Nothing
notThe str   = Just str

replaceThe :: String -> String
replaceThe str = 
    let parts = words str
        replace = maybe "a" id
        replaced = map (replace . notThe) parts
    in unwords replaced
    
replaceThe "cat in the hat"

"cat in a hat"

In [2]:
isVowel :: Char -> Bool
isVowel c = elem c [ 'a' , 'e' , 'i' , 'o' , 'u' ]

countTheBeforeVowel :: String -> Integer
countTheBeforeVowel str = count (words str)
    where count []                       = 0
          count [_]                      = 0
          count ("the" : ((c:_) : rest)) = (if isVowel c then 1 else 0) + (count rest)
          count (_ : rest)               = count rest

countTheBeforeVowel "the cow"
countTheBeforeVowel "the cow eats"
countTheBeforeVowel "the cow eats the grass"
countTheBeforeVowel "the evil eats the grass"
countTheBeforeVowel "the evil eats the emo"

0

0

0

1

2

In [3]:
countVowels :: String -> Int
countVowels = length . (filter isVowel)

countVowels "life"

2

In [4]:
newtype Word' = Word' String deriving (Eq, Show)

In [5]:
isConsonant :: Char -> Bool
isConsonant c = elem c ['a'..'z'] && (not $ isVowel c) 

countConsonants :: String -> Int
countConsonants = length . (filter isConsonant)

mkWord :: String -> Maybe Word'
mkWord str | countVowels str <= countConsonants str = Just (Word' str)
           | otherwise                              = Nothing

mkWord "hello"
mkWord "aaaa"

Just (Word' "hello")

Nothing

In [6]:
data Nat = Zero | Succ Nat deriving (Eq, Show)

natToInteger :: Nat -> Integer
natToInteger Zero = 0
natToInteger (Succ nat) = (+1) (natToInteger nat)

natToInteger (Succ (Succ Zero))

integerToNat :: Integer -> Maybe Nat
integerToNat n | n < 0     = Nothing
               | otherwise = Just (int2Nat n) 
                   where int2Nat 0 = Zero
                         int2Nat n = Succ (int2Nat (n-1))
                         
integerToNat 3

2

Just (Succ (Succ (Succ Zero)))

In [7]:
isJust :: Maybe a -> Bool
isJust (Just _) = True
usJust _        = False

isNothing :: Maybe a -> Bool
isNothing m = not (isJust m)

mayybee :: b -> (a -> b) -> Maybe a -> b
mayybee ifNothing ifSomething m = case m of
    Just something -> ifSomething something
    Nothing        -> ifNothing
    
mayybee 1 (+3) (Just 9)
mayybee 1 (+3) Nothing

fromMaybe :: a -> Maybe a -> a
fromMaybe def = mayybee def id

fromMaybe 1 (Just 3)
fromMaybe 1 Nothing

listToMaybe :: [a] -> Maybe a
listToMaybe (a:_) = Just a
listToMaybe _     = Nothing

maybeToList :: Maybe a -> [a]
maybeToList = mayybee [] (:[])

maybeToList (Just 42)
maybeToList Nothing

catMaybes :: [Maybe a] -> [a]
catMaybes [] = []
catMaybes (Just a : tail) = a : catMaybes tail
catMaybes (Nothing: tail) = catMaybes tail

catMaybes [Just 1, Nothing, Just 3]

flipMaybe :: [Maybe a] -> Maybe [a]
flipMaybe [] = Just []
flipMaybe (ma : tail) = case ma of
    Just a  -> case flipMaybe tail of
        Just tail -> Just (a : tail)
        Nothing   -> Nothing
    Nothing -> Nothing
    
flipMaybe [Just 1, Nothing]
flipMaybe [Just 1, Just 3]
flipMaybe [Just 1, Just 3, Nothing]

12

1

3

1

[42]

[]

[1,3]

Nothing

Just [1,3]

Nothing

In [8]:
lefts' :: [Either a b] -> [a]
lefts' = foldr f []
    where  f (Right _) rest = rest
           f (Left  a) rest = a : rest
           
lefts' [Left 1, Right 2, Left 3]

rights' :: [Either a b] -> [b]
rights' = foldr f []
    where  f (Right b) rest = b : rest
           f (Left  _) rest = rest
           
partitionEithers' :: [Either a b] -> ([a], [b])
partitionEithers' = foldr f ([],[])
    where f (Right b) (ls,rs) = (ls,b:rs)
          f (Left  a) (ls,rs) = (a:ls,rs)
          
partitionEithers' [Right 1, Left "a", Right 2, Right 3, Left "b"]

either' :: (a -> c) -> (b -> c) -> Either a b -> c
either' left _  (Left a ) = left a
either' _ right (Right b) = right b

eitherMaybe'' :: (b -> c) -> Either a b -> Maybe c
eitherMaybe'' f = either' (const Nothing) (Just . f)

[1,3]

(["a","b"],[1,2,3])

In [9]:
myIterate :: (a -> a) -> a -> [a]
myIterate f a0 = a0 : (myIterate f (f a0))

take 5 $ myIterate (+1) 0

myUnfoldr :: (b -> Maybe (a, b)) -> b -> [a]
myUnfoldr f b = maybe [] (\(a,next) -> a : myUnfoldr f next) (f b)

betterIterate :: (a -> a) -> a -> [a]
betterIterate f = myUnfoldr (\a -> Just (a, f a))

take 5 $ betterIterate (+1) 0


[0,1,2,3,4]

[0,1,2,3,4]

In [10]:
data BinaryTree a = Leaf | Node (BinaryTree a) a (BinaryTree a) deriving (Eq, Ord, Show)

unfold :: (a -> Maybe (a,b,a)) -> a -> BinaryTree b
unfold f a0 = maybe Leaf (\(l,a,r) -> Node (unfold f l) a (unfold f r)) (f a0)

treeBuild :: Integer -> BinaryTree Integer
treeBuild n = unfold f 0
    where f x | x == n = Nothing
              | otherwise = Just (x+1,x,x+1)

treeBuild 0
treeBuild 1
treeBuild 2
treeBuild 3

Leaf

Node Leaf 0 Leaf

Node (Node Leaf 1 Leaf) 0 (Node Leaf 1 Leaf)

Node (Node (Node Leaf 2 Leaf) 1 (Node Leaf 2 Leaf)) 0 (Node (Node Leaf 2 Leaf) 1 (Node Leaf 2 Leaf))